In [1]:
import pandas as pd 
data = pd.read_excel("dataset.xlsx", sheet_name="Run 1_ PV_battery_model")
data= data.drop(["time" , "Snfc"] , axis=1)
datax = data.drop(columns=['Snet'])
dfy1 = data['Snet']

In [2]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(datax)
dfx1=scaler.transform(datax)
dfx1 = pd.DataFrame(dfx1)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dfx1,dfy1,test_size=0.2, random_state=42) 

# Blue Whale Optimization

In [4]:
import numpy as np
import random
from sklearn.metrics import root_mean_squared_error
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from joblib import Parallel, delayed

# Individual Whale
class Whale:
    def __init__(self, dim, bounds):
        self.position = np.array([random.uniform(bounds[d][0], bounds[d][1]) for d in range(dim)])
        self.fitness = float('inf')

# BWO Algorithm
def blue_whale_optimize(func, dim, bounds, n_whales=7, max_iter=20, a_decay=2, early_stop=5):
    whales = [Whale(dim, bounds) for _ in range(n_whales)]
    best_whale = min(whales, key=lambda w: func(w.position))
    best_loss = func(best_whale.position)
    no_improve = 0

    for iter in range(max_iter):
        a = 2 - iter * (2 / max_iter)  # Linearly decreasing a from 2 to 0

        scores = Parallel(n_jobs=-1)(delayed(func)(w.position) for w in whales)

        for i, whale in enumerate(whales):
            fitness = scores[i]

            if fitness < whale.fitness:
                whale.fitness = fitness

            if fitness < best_loss:
                best_loss = fitness
                best_whale = Whale(dim, bounds)
                best_whale.position = whale.position.copy()
                best_whale.fitness = fitness
                no_improve = 0
            else:
                no_improve += 1

            if no_improve >= early_stop:
                return best_whale.position, best_loss

        for whale in whales:
            r = random.random()
            A = 2 * a * r - a
            C = 2 * r

            if random.random() < 0.5:
                D = np.abs(C * best_whale.position - whale.position)
                whale.position = best_whale.position - A * D
            else:
                b = 1
                l = random.uniform(-1, 1)
                distance_to_best = np.abs(best_whale.position - whale.position)
                whale.position = distance_to_best * np.exp(b * l) * np.cos(2 * np.pi * l) + best_whale.position

            whale.position = np.clip(whale.position, [b[0] for b in bounds], [b[1] for b in bounds])

    return best_whale.position, best_loss

# Regression + BWO wrapper
def regression_bwo(model_class, param_bounds, X_train, X_test, y_train, y_test, n_whales=7, max_iter=12):
    def model_fitness(params):
        params = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), params)}
        model = model_class(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return root_mean_squared_error(y_test, y_pred)  # RMSE

    bounds = list(param_bounds.values())
    best_params, best_loss = blue_whale_optimize(model_fitness, len(bounds), bounds, n_whales, max_iter)

    best_params_dict = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), best_params)}
    return best_params_dict, best_loss

# Example Usage
if __name__ == "__main__":
    # Example dataset

    models = {
        "GradientBoosting": (GradientBoostingRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "XGBRegressor": (XGBRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "RandomForest": (RandomForestRegressor, {"n_estimators": (50, 100), "max_depth": (1, 20)}),
        "DecisionTree": (DecisionTreeRegressor, {"max_depth": (1, 20), "min_samples_split": (2, 10)}),
    }

    for name, (model_class, param_bounds) in models.items():
        print(f"\nOptimizing {name} using Blue Whale Optimization...")
        best_params, best_loss = regression_bwo(model_class, param_bounds, X_train, X_test, y_train, y_test)
        print(f"Best Parameters: {best_params}, Best Loss (RMSE): {best_loss:.4f}")



Optimizing GradientBoosting using Blue Whale Optimization...
Best Parameters: {'n_estimators': 81, 'learning_rate': 0.19071380620781497}, Best Loss (RMSE): 1.3612

Optimizing XGBRegressor using Blue Whale Optimization...
Best Parameters: {'n_estimators': 84, 'learning_rate': 0.17830877471388534}, Best Loss (RMSE): 0.3436

Optimizing RandomForest using Blue Whale Optimization...
Best Parameters: {'n_estimators': 80, 'max_depth': 18}, Best Loss (RMSE): 0.2131

Optimizing DecisionTree using Blue Whale Optimization...
Best Parameters: {'max_depth': 15, 'min_samples_split': 2}, Best Loss (RMSE): 0.2639


# Ant Colony Optimization (ACO)  is for ANN


In [4]:
import numpy as np
import random
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

# Ant Colony Optimization for ANN
def ant_colony_optimize(func, param_bounds, n_ants=10, n_iterations=20, evaporation_rate=0.3, Q=1.0):
    dim = len(param_bounds)
    pheromones = np.ones((dim, 100))  # 100 discretization bins per dimension
    bounds = list(param_bounds.values())
    keys = list(param_bounds.keys())

    best_solution = None
    best_score = float('inf')

    def decode_position(position):
        decoded = []
        for i in range(dim):
            low, high = bounds[i]
            bin_index = int(position[i])
            val = low + (high - low) * bin_index / 99
            if isinstance(low, int):
                val = int(round(val))
            decoded.append(val)
        return decoded

    for _ in range(n_iterations):
        ants = []
        scores = []

        for _ in range(n_ants):
            position = []
            for d in range(dim):
                probabilities = pheromones[d] / pheromones[d].sum()
                choice = np.random.choice(100, p=probabilities)
                position.append(choice)
            decoded = decode_position(position)
            score = func(decoded)
            ants.append(position)
            scores.append(score)

            if score < best_score:
                best_score = score
                best_solution = decoded

        # Pheromone update
        pheromones *= (1 - evaporation_rate)
        for i in range(n_ants):
            for d in range(dim):
                pheromones[d, ants[i][d]] += Q / (scores[i] + 1e-6)

    best_params = {key: best_solution[i] for i, key in enumerate(keys)}
    return best_params, best_score

# ANN Regression using ACO
def ann_aco(X_train, X_test, y_train, y_test):
    param_bounds = {
        "hidden_layer_sizes": (10, 200),  # number of neurons in one layer
        "alpha": (1e-5, 1e-1),            # L2 penalty
        "learning_rate_init": (1e-4, 1e-1)
    }

    def model_fitness(params):
        # convert hidden_layer_sizes to tuple for sklearn
        hidden_layer_sizes = (int(params[0]),)
        alpha = float(params[1])
        learning_rate = float(params[2])

        model = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes,
                             alpha=alpha,
                             learning_rate_init=learning_rate,
                             max_iter=500,
                             random_state=42)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return root_mean_squared_error(y_test, y_pred, squared=False)  # RMSE

    best_params, best_loss = ant_colony_optimize(model_fitness, param_bounds)
    return best_params, best_loss

# Example Usage
if __name__ == "__main__":
    # Generate regression dataset
   
    print("Running ACO to optimize ANN hyperparameters...")
    best_params, best_loss = ann_aco(X_train, X_test, y_train, y_test)
    print("\nBest Parameters:")
    print(best_params)
    print(f"Best RMSE: {best_loss:.4f}")


Running ACO to optimize ANN hyperparameters...


TypeError: got an unexpected keyword argument 'squared'

# Cuckoo Search for Hyperparameter Optimization 

In [6]:
import numpy as np
import random
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

# Lévy flight implementation
def levy_flight(Lambda):
    sigma = (np.math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) / 
            (np.math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
    u = np.random.randn() * sigma
    v = np.random.randn()
    step = u / abs(v) ** (1 / Lambda)
    return step

# Cuckoo Search main function
def cuckoo_search(func, bounds, n_nests=15, max_iter=20, pa=0.25):
    dim = len(bounds)
    nests = [np.array([random.uniform(bounds[d][0], bounds[d][1]) for d in range(dim)]) for _ in range(n_nests)]
    fitness = [func(nest) for nest in nests]
    best_nest = nests[np.argmin(fitness)]
    best_fitness = min(fitness)

    for _ in range(max_iter):
        for i in range(n_nests):
            # Generate new solution via Lévy flight
            new_nest = nests[i] + 0.01 * np.array([levy_flight(1.5) for _ in range(dim)])
            new_nest = np.clip(new_nest, [b[0] for b in bounds], [b[1] for b in bounds])
            new_fitness = func(new_nest)

            if new_fitness < fitness[i]:
                nests[i] = new_nest
                fitness[i] = new_fitness

        # Abandon a fraction (pa) of worse nests
        num_abandon = int(pa * n_nests)
        for _ in range(num_abandon):
            i = random.randint(0, n_nests - 1)
            nests[i] = np.array([random.uniform(bounds[d][0], bounds[d][1]) for d in range(dim)])
            fitness[i] = func(nests[i])

        current_best = nests[np.argmin(fitness)]
        current_best_fitness = min(fitness)
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            best_nest = current_best

    return best_nest, best_fitness

# Regression wrapper
def regression_cuckoo(model_class, param_bounds, X_train, X_test, y_train, y_test, n_nests=10, max_iter=15):
    def model_fitness(params):
        param_dict = {key: int(val) if isinstance(param_bounds[key][0], int) else val 
                      for key, val in zip(param_bounds.keys(), params)}
        model = model_class(**param_dict)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return root_mean_squared_error(y_test, y_pred)

    bounds = list(param_bounds.values())
    best_params_list, best_loss = cuckoo_search(model_fitness, bounds, n_nests, max_iter)

    best_params = {key: int(val) if isinstance(param_bounds[key][0], int) else val 
                   for key, val in zip(param_bounds.keys(), best_params_list)}
    return best_params, best_loss

# Example usage
if __name__ == "__main__":
   
    models = {
        "GradientBoosting": (GradientBoostingRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "XGBRegressor": (XGBRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        # "RandomForest": (RandomForestRegressor, {"n_estimators": (50, 100), "max_depth": (1, 20)}),
        # "DecisionTree": (DecisionTreeRegressor, {"max_depth": (1, 20), "min_samples_split": (2, 10)}),
    }

    for name, (model_class, param_bounds) in models.items():
        print(f"\nOptimizing {name} using Cuckoo Search...")
        best_params, best_loss = regression_cuckoo(model_class, param_bounds, X_train, X_test, y_train, y_test)
        print(f"Best Parameters: {best_params}")
        print(f"Best RMSE: {best_loss:.4f}")



Optimizing GradientBoosting using Cuckoo Search...


C:\Users\dsmat\AppData\Local\Temp\ipykernel_4548\1279342632.py:12: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  sigma = (np.math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) /
C:\Users\dsmat\AppData\Local\Temp\ipykernel_4548\1279342632.py:13: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  (np.math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
C:\Users\dsmat\AppData\Local\Temp\ipykernel_4548\1279342632.py:12: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  sigma = (np.math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) /
C:\Users\dsmat\AppData\Local\Temp\ipykernel_4548\1279342632.py:13: DeprecationWarning: `np.math` is a depre

KeyboardInterrupt: 